In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [2]:
inp=pd.read_csv("../PJ sensor.csv",usecols=[6,7,10,11])
out=pd.read_csv("../PJ sensor.csv",usecols=[2,3,4,5,8,9])

In [4]:
inp.head(5)

,Temperature,Relative Humidity,date,Time
0,31,65,191213,175844
1,31,65,191213,175904
2,31,65,191213,175924
3,31,65,191213,175944
4,31,66,191213,180004


In [5]:
out.head(5)

,NO2,O3,CO,SO2,PM2.5,PM10
0,-15.075377,35.714286,1766.233766,148.484848,11.229150,17.323938
1,-20.100503,41.208791,1733.766234,142.424242,9.413914,24.967407
2,-15.075377,43.956044,1711.038961,136.363636,10.350110,56.781536
3,0.000000,57.692308,1688.311688,139.393939,10.488765,30.371115
4,-20.100503,54.945055,1681.818182,130.303030,12.353048,188.026886


In [3]:
inp=np.array(inp)
out=np.array(out)

In [4]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(inp)
Y1=scaler_obj.fit_transform(out)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [7]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(RandomForestRegressor(max_depth=10,n_estimators=100, random_state=0))

model_fit1=model1.fit(x_train, y_train)


print("Model training is Done!!")

filename1 = 'random.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

Model training is Done!!


In [8]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')


r2 score on train data 0.6406985506761919
r2 score on test data 0.6329507787480243
Mean Absolute Error: 0.009558281283174914
Mean Squared Error: 0.00029808595786701435
Root Mean Squared Error: 0.017265166024889953
 



In [5]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("random.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[0.47446701 0.59809331 0.04642474 0.00860868 0.06751412 0.00456361]
 [0.4079212  0.55555687 0.03019471 0.00751099 0.04082953 0.00398102]
 [0.48788127 0.59818488 0.05305332 0.00954183 0.11863581 0.0063403 ]
 ...
 [0.41762684 0.56264799 0.0301981  0.00983107 0.06200965 0.00533818]
 [0.39566302 0.55127623 0.03061972 0.01038668 0.0661479  0.00511552]
 [0.4674526  0.55501368 0.04413879 0.00825391 0.05801073 0.00472562]]


R2 Score :  0.6329507787480243


In [6]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=r2_score(y_test[:,i],y_test_pred[:,i])
    print("R2 Score of "+train_sizes[i]+":",temp)

R2 Score of NO2: 0.7105192952762133
R2 Score of O3: 0.4903772580751794
R2 Score of NO: 0.611726541293893
R2 Score of CO: 0.7508022653342522
R2 Score of PM2.5: 0.6709906432263035
R2 Score of PM10: 0.2534653978921255


In [7]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=metrics.mean_squared_error(y_test[:,i],y_test_pred[:,i])
    print("MSE Value of "+train_sizes[i]+":",temp)

MSE Value of NO2: 0.0005542341514946809
MSE Value of O3: 0.0006817994662347451
MSE Value of NO: 7.449939582899556e-05
MSE Value of CO: 7.554462934596531e-07
MSE Value of PM2.5: 0.0004624333836156638
MSE Value of PM10: 1.4793903734520713e-05


In [8]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=np.sqrt(metrics.mean_squared_error(y_test[:,i],y_test_pred[:,i]))
    print("RMSE Value of "+train_sizes[i]+":",temp)

RMSE Value of NO2: 0.023542178138283656
RMSE Value of O3: 0.026111290014756934
RMSE Value of NO: 0.008631303252058495
RMSE Value of CO: 0.0008691641349363497
RMSE Value of PM2.5: 0.021504264312355907
RMSE Value of PM10: 0.0038462844063486402


In [9]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=metrics.mean_absolute_error(y_test[:,i],y_test_pred[:,i])
    print("MAE Value of "+train_sizes[i]+":",temp)

MAE Value of NO2: 0.0170942904157586
MAE Value of O3: 0.017157796175211328
MAE Value of NO: 0.005502722016153936
MAE Value of CO: 0.0005737368563099131
MAE Value of PM2.5: 0.01481065960192509
MAE Value of PM10: 0.0022104826336923414


In [ ]:
#completed